In [1]:
import gzip
import numpy as np
from collections import defaultdict

def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

### Rating baseline: compute averages for each user, or return the global average if we've never seen the user before

allRatings = []
userRatings = defaultdict(list)
for l in readGz("train.json.gz"):
    user,business = l['userID'],l['businessID']
    allRatings.append(l['rating'])
    userRatings[user].append(l['rating'])

globalAverage = sum(allRatings) / len(allRatings)
userAverage = {}
for u in userRatings:
    userAverage[u] = sum(userRatings[u]) / len(userRatings[u])

predictions = open("predictions_Rating.txt", 'w')
for l in open("pairs_Rating.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    if u in userAverage:
        predictions.write(u + '-' + i + ',' + str(userAverage[u]) + '\n')
    else:
        predictions.write(u + '-' + i + ',' + str(globalAverage) + '\n')

predictions.close()

In [2]:
### Would-visit baseline: just rank which businesses are popular and which are not, and return '1' if a business is among the top-ranked
businessCount = defaultdict(int)
totalPurchases = 0

for l in readGz("train.json.gz"):
    user,business = l['userID'],l['businessID']
    businessCount[business] += 1
    totalPurchases += 1

mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalPurchases/2: break

predictions = open("predictions_Visit.txt", 'w')
for l in open("pairs_Visit.txt"):
    if l.startswith("userID"):
    #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    if i in return1:
        predictions.write(u + '-' + i + ",1\n")
    else:
        predictions.write(u + '-' + i + ",0\n")

predictions.close()

In [3]:
### Category prediction baseline: Just consider some of the most common words from each category

catDict = {
  "American Restaurant": 0,
  "Bar": 1,
  "Asian Restaurant": 2,
  "European Restaurant": 3,
  "Italian Restaurant": 4,
  "Fast Food Restaurant": 5,
  "Mexican Restaurant": 6,
  "Seafood Restaurant": 7,
  "Coffee Shop": 8,
  "Sandwich Shop": 9
}

predictions = open("predictions_Category.txt", 'w')
predictions.write("userID-reviewHash,category\n")
for l in readGz("test_Category.json.gz"):
    cat = catDict['American Restaurant'] # If there's no evidence, just choose the most common category in the dataset
    words = l['reviewText'].lower()
    if 'america' in words:
        cat = catDict['American Restaurant']
    if 'bar' in words or 'beer' in words:
        cat = catDict['Bar']
    if 'asia' in words:
        cat = catDict['Asian Restaurant']
    if 'europe' in words:
        cat = catDict['European Restaurant']
    if 'italian' in words:
        cat = catDict['Italian Restaurant']
    if 'fast' in words:
        cat = catDict['Fast Food Restaurant']
    if 'mexic' in words:
        cat = catDict['Mexican Restaurant']
    if 'coffee' in words:
        cat = catDict['Coffee Shop']
    if 'sandwich' in words:
        cat = catDict['Sandwich Shop']
    predictions.write(l['userID'] + '-' + l['reviewHash'] + "," + str(cat) + "\n")

predictions.close()


In [42]:
########Q1
import gzip
import numpy as np
from collections import defaultdict
def readGz(f):
    for l in gzip.open(f):
        yield eval(l)
data=list(readGz("train.json.gz"))
trainset=data[0:100000]
validationset=data[100000:200000]

In [69]:
businesslist = []
userlist=[]
for l in data:
    user,business = l['userID'],l['businessID']
    businesslist.append(business)
    userlist.append(user)
businesslist=list(set(businesslist))  
userlist=list(set(userlist))   


In [71]:
businessIDlist={}
userIDlist={}
for i in range(len(businesslist)):
    businessIDlist[businesslist[i]]=i
for j in range(len(userlist)):
    userIDlist[userlist[j]]=j
    

In [72]:
data1=[]
temp=[]
for l in data:
    dic={}
    user,business = l['userID'],l['businessID']
    dic['userID']=userIDlist[user]
    dic['businessID']=businessIDlist[business]
    dic['isvisit']=1
    data1.append(dic)
    temp.append([userIDlist[user],businessIDlist[business] ])

In [77]:
###random create validation set
import random
validset=[]
while len(validset)<100000:
    dic={}
    i=random.randint(0,len(userlist)-1)
    j=random.randint(0,len(businesslist)-1)
    temp1=[i,j]
    if temp1 not in temp:
        dic['userID']=i
        dic['businessID']=j
        dic['isvisit']=0
        validset.append(dic)
#combine validset
valid=data1[100000:200000]+validset        
    

In [78]:
import pandas as pd
traintotal=data1+validset     
df=pd.DataFrame(traintotal)


In [164]:
businessCount = defaultdict(int)
totalPurchases = 0

for l in readGz("train.json.gz"):
    user,business = l['userID'],l['businessID']
    businessCount[business] += 1
    totalPurchases += 1

mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalPurchases/2: break

In [88]:
X_train=df.drop()

In [99]:
X_train = df.drop('isvisit',axis=1)
y_train = df['isvisit']

In [100]:
X_train=X_train.as_matrix()
y_train=y_train.as_matrix()


In [102]:
def compute_distances_no_loops(X_train, X):
    num_test = X.shape[0]
    num_train =X_train.shape[0]
    dists = np.zeros((num_test, num_train)) 
    dists = np.sqrt((X**2).sum(axis=1, keepdims=True) + (X_train**2).sum(axis=1) - 2 * X.dot(X_train.T))
    return dists
def predict_labels(y_train, dists, k=1):
    num_test = dists.shape[0]
    y_pred = np.zeros(num_test)
    for i in xrange(num_test):     
        closest_y = []
        closest_y = y_train[np.argsort(dists[i])][:k] 
        y_pred[i] = np.argmax(np.bincount(closest_y))
    return y_pred

In [169]:
predictions = open("predictions_Visit.txt", 'w')
for l in open("pairs_Visit.txt"):
    if l.startswith("userID"):
    #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
#     if i in return1:
#         predictions.write(u + '-' + i + ",1\n")
#     else:
#         predictions.write(u + '-' + i + ",0\n")
    if u in userIDlist and i in businessIDlist:
        u_id=userIDlist[u]
        i_id=businessIDlist[i]
        x_test=np.array((i_id,u_id))
        result=neigh.predict([x_test])
        result=str(int(result[0]))
    elif i in return1:
        result="1"
    else:
        result="0"
    predictions.write(u + '-' + i + ',' + result + '\n')
predictions.close()

In [168]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train, y_train) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [189]:
data2=[]
for l in data:
    dic={}
    user,business,rating = l['userID'],l['businessID'],l["rating"]
    dic['userID']=userIDlist[user]
    dic['rating']=rating
    dic['businessID']=businessIDlist[business]
    dic['isvisit']=1
    data2.append(dic)
    #temp.append([userIDlist[user],businessIDlist[business] ])

In [192]:
from sklearn import cross_validation as cv
df2=pd.DataFrame(data2)

n_users = df2.userID.unique().shape[0]
n_business=df2.businessID.unique().shape[0]
train_data, test_data = cv.train_test_split(df2, test_size=0.25)

#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_business))

In [232]:
business1=train_data["businessID"].as_matrix()
user1=train_data["userID"].as_matrix()
rating1=train_data["rating"].as_matrix()
business2=test_data["businessID"].as_matrix()
user2=test_data["userID"].as_matrix()
rating2=test_data["rating"].as_matrix()

In [253]:
business1[0],user1[0]

(12728, 13578)

In [240]:
for l in range(len(train_data)):
    train_data_matrix[user1[l], business1[l]] = rating1[l]
test_data_matrix = np.zeros((n_users, n_business))
for l in range(len(test_data)):
    test_data_matrix[user2[l], business2[l]] = rating2[l]

In [243]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')

In [260]:
def predict(ratings, similarity, type='user'):
        if type == 'user':
            mean_user_rating = ratings.mean(axis=1)
            #print mean_user_rating
            ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
            pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T   
            return pred

In [249]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
        prediction = prediction[ground_truth.nonzero()].flatten() 
        ground_truth = ground_truth[ground_truth.nonzero()].flatten()
        return sqrt(mean_squared_error(prediction, ground_truth))
                
print 'User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix))

User-based CF RMSE: 4.27361586833


In [276]:
#######
userRatings = defaultdict(lambda: defaultdict(int))
businessRatings_train = defaultdict(lambda: defaultdict(int))
for l in data:
    user,business,rating = l['userID'],l['businessID'],l['rating']
    userRatings[user][business] = rating
    businessRatings_train[business][user] = rating
    

In [325]:
def similarity_score(person1,person2,userRatings):
    
    # Returns ratio Euclidean distance score of person1 and person2 

    both_viewed = {}# To get both rated items by person1 and person2

    for item in userRatings[person1]:
        if item in userRatings[person2]:
            both_viewed[item] = 1

        # Conditions to check they both have an common rating items	
    if len(both_viewed) == 0:
        return 0

    # Finding Euclidean distance 
    sum_of_eclidean_distance = []
    for item in userRatings[person1]:
        if item in userRatings[person2]:
            sum_of_eclidean_distance.append(pow(userRatings[person1][item] - userRatings[person2][item],2))
    sum_of_eclidean_distance = sum(sum_of_eclidean_distance)
    #print sum_of_eclidean_distance

    return 1/(1+sqrt(sum_of_eclidean_distance))*1.0

In [305]:
both_viewed = {}# To get both rated items by person1 and person2
for item in userRatings["U024954695"]:
    if item in userRatings["U058381151"]:
        both_viewed[item] = 1
both_viewed

{'B807454692': 1}

In [332]:
def pearson_correlation(person1,person2,userRatings):

    # To get both rated items
    both_rated = {}
    for item in userRatings[person1]:
        if item in userRatings[person2]:
            both_rated[item] = 1

    number_of_ratings = len(both_rated)
    
    # Checking for number of ratings in common
    if number_of_ratings == 0:
        return 0

    # Add up all the preferences of each user
    person1_preferences_sum = sum([userRatings[person1][item] for item in both_rated])
    person2_preferences_sum = sum([userRatings[person2][item] for item in both_rated])

    # Sum up the squares of preferences of each user
    person1_square_preferences_sum = sum([pow(userRatings[person1][item],2) for item in both_rated])
    person2_square_preferences_sum = sum([pow(userRatings[person2][item],2) for item in both_rated])

    # Sum up the product value of both preferences for each item
    product_sum_of_both_users = sum([userRatings[person1][item] * userRatings[person2][item] for item in both_rated])

    # Calculate the pearson score
    numerator_value = product_sum_of_both_users - (person1_preferences_sum*person2_preferences_sum/number_of_ratings)
    denominator_value = sqrt((person1_square_preferences_sum - pow(person1_preferences_sum,2)/number_of_ratings) * (person2_square_preferences_sum -pow(person2_preferences_sum,2)/number_of_ratings))
    if denominator_value == 0:
        return 0
    else:
        r = numerator_value/denominator_value
        return r 

In [460]:
def pearson_correlation1(person1,person2,userRatings):

    # To get both rated items
    both_rated = {}
    item1={}
    item2={}

    for item in userRatings[person1]:
        item1[item]=1
        if item in userRatings[person2]:
            both_rated[item] = 1
    for item in userRatings[person2]:
        item2[item]=1
    number_of_ratings = len(both_rated)
    
    # Checking for number of ratings in common
    if number_of_ratings == 0:
        return 0


    #average score for user 1 and user2
    user1_bar=sum([userRatings[person1][item] for item in item1])/len(item1)
    user2_bar=sum([userRatings[person2][item] for item in item2])/len(item2)
    
    #####
    top=sum([(userRatings[person1][item]-user1_bar)*(userRatings[person2][item]-user2_bar) for item in both_rated])
    #####
    bottom1= sqrt(sum([pow((userRatings[person1][item]-user1_bar),2) for item in both_rated]))
    bottom2= sqrt(sum([pow((userRatings[person2][item]-user2_bar),2) for item in both_rated]))
    if bottom1*bottom2==0:
        return 0
    else:
        return top/(bottom1*bottom2)
    

                  
    

In [434]:
def most_similar_users(person,number_of_users):
	# returns the number_of_users (similar persons) for a given specific person.
	scores = [(pearson_correlation1(person,other_person,userRatings),other_person) for other_person in userRatings if  other_person != person ]
	
	# Sort the similar persons so that highest scores person will appear at the first
	scores.sort()
	scores.reverse()
	return scores[0:number_of_users]

In [435]:
print most_similar_users('U058381151',100)

[(1.0, 'U998018573'), (1.0, 'U988807085'), (1.0, 'U964717138'), (1.0, 'U955976623'), (1.0, 'U954605752'), (1.0, 'U938695606'), (1.0, 'U935602736'), (1.0, 'U931553259'), (1.0, 'U923196285'), (1.0, 'U908745071'), (1.0, 'U898364001'), (1.0, 'U878954232'), (1.0, 'U851278078'), (1.0, 'U842112328'), (1.0, 'U830025839'), (1.0, 'U806898128'), (1.0, 'U803012061'), (1.0, 'U801666563'), (1.0, 'U801328664'), (1.0, 'U787087972'), (1.0, 'U784218536'), (1.0, 'U772009262'), (1.0, 'U770044015'), (1.0, 'U768126864'), (1.0, 'U746613760'), (1.0, 'U739287075'), (1.0, 'U737881024'), (1.0, 'U735808692'), (1.0, 'U721124121'), (1.0, 'U719203803'), (1.0, 'U705372537'), (1.0, 'U696414886'), (1.0, 'U694597071'), (1.0, 'U688802674'), (1.0, 'U683277484'), (1.0, 'U681872193'), (1.0, 'U680674395'), (1.0, 'U680319518'), (1.0, 'U679140107'), (1.0, 'U674664964'), (1.0, 'U673400876'), (1.0, 'U673182190'), (1.0, 'U671462264'), (1.0, 'U671366769'), (1.0, 'U668448539'), (1.0, 'U667710535'), (1.0, 'U657217729'), (1.0, 'U6543

In [461]:
def user_reommendations(person):

	# Gets recommendations for a person by using a weighted average of every other user's rankings
	totals = {}
	simSums = {}
	rankings_list =[]
	for other in userRatings:
		# don't compare me to myself
		if other == person:
			continue
		sim = pearson_correlation1(person,other,userRatings)
		#print ">>>>>>>",sim

		# ignore scores of zero or lower
		if sim <=0: 
			continue
		for item in userRatings[other]:

			# only score movies i haven't seen yet
			if item not in userRatings[person] or userRatings[person][item] == 0:

			# Similrity * score
				totals.setdefault(item,0)
				totals[item] += userRatings[other][item]* sim
				# sum of similarities
				simSums.setdefault(item,0)
				simSums[item]+= sim

		# Create the normalized list

	rankings = [(total/simSums[item],item) for item,total in totals.items()]
	rankings.sort()
	rankings.reverse()
	# returns the recommended items
	recommendataions_list = [recommend_item for score,recommend_item in rankings]
	return recommendataions_list

In [465]:
predictions = open("predictions_Visit.txt", 'w')
for l in open("pairs_Visit.txt"):
    if l.startswith("userID"):
    #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    res=0
    if u in userIDlist and i in businessIDlist:
        business=user_reommendations(u)
        if i in business:
            res=1
            #predictions.write(u + '-' + i + ",1\n")
#         else:
#             for cat in (categorydic[i]):
#                 for j in range(len(usercategorydic[u])):
#                     if cat in usercategorydic[u][j]:
#                          res=1
    else:
        for cat in (categorydic[i]):
            for j in range(len(usercategorydic[u])):
                if cat in usercategorydic[u][j]:
                     res=1
    if res==1:
        predictions.write(u + '-' + i + ",1\n")
    else:
        predictions.write(u + '-' + i + ",0\n")
predictions.close()

In [336]:
both_rated = {}
for item in userRatings["U058381151"]:
    if item in userRatings["U024954695"]:
        both_rated[item] = 1

number_of_ratings = len(both_rated)
number_of_ratings
person1_preferences_sum = sum([userRatings["U058381151"][item] for item in both_rated])
person2_preferences_sum = sum([userRatings["U024954695"][item] for item in both_rated])

In [352]:
person1_square_preferences_sum = sum([pow(userRatings["U058381151"][item],2) for item in both_rated])
person2_square_preferences_sum = sum([pow(userRatings["U024954695"][item],2) for item in both_rated])
product_sum_of_both_users = sum([userRatings["U058381151"][item] * userRatings["U024954695"][item] for item in both_rated])
numerator_value = product_sum_of_both_users - (person1_preferences_sum*person2_preferences_sum/number_of_ratings)
denominator_value = sqrt((person1_square_preferences_sum - pow(person1_preferences_sum,2)/number_of_ratings) * (person2_square_preferences_sum -pow(person2_preferences_sum,2)/number_of_ratings))


In [11]:
##calculate accuracy of baseline on validation set
num=0
for l in valid:
    u=l['businessID']
    if u in return1:
        y=1
    else:
        y=0
    if y==l['isvisit']:  #count the correct pairs
        num+=1
acc=num*1.0/len(valid)
print "when threshold is 50th percentile"
print "accuracy is",acc

when threshold is 50th percentile
accuracy is 0.650585


In [30]:
# change threshold from 50th percentile of popularity to 67th percentile of popularity
return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalPurchases/1.5: break
##calculate accuracy of baseline on validation set
num=0
for l in valid:
    u=l['businessID']
    if u in return1:
        y=1
    else:
        y=0
    if y==l['isvisit']:  #count the correct pairs
        num+=1
acc=num*1.0/len(valid)
print "when threshold is 67th percentile"
print "accuracy is",acc

when threshold is 67th percentile
accuracy is 0.653975


I change the threshold from 50th percentile of popularity to 67th percentile of popularity. The accuracy slightly increases.

In [390]:
#####Q3
usercategorydic=defaultdict(list)
categorydic = defaultdict(list)
for l in data:
    user,business,categories,rating = l['userID'],l['businessID'],l['categories'],l['rating']
    usercategorydic[user].append(categories)
    categorydic[business]=(categories)


In [26]:
y=[1]*100000+[0]*100000 #create y for valid set
y_pred=[0]*len(valid)
for i,l in enumerate(valid):
    user,business = l['userID'],l['businessID']
    #print user,business
    if type(usercategorydic[user])!=int and type(categorydic[business])!=int:
        for cat in (categorydic[business]):
            for i in range(len(usercategorydic[user])):
                if cat in usercategorydic[user][i]:
                    y_pred[i]=1  #calculate y_pred
                

In [22]:
#calculate accuracy
correct = [(a==b) for (a,b) in zip(y_pred,y)]
acc = sum(correct) * 1.0 / len(correct)
print "accuracy is",acc


accuracy is 0.502855


In [177]:
#####Q4 input the model to pairs_Visit.txt and submit on Kaggle
predictions = open("predictions_Visit.txt", 'w')
c=0
for l in open("pairs_Visit.txt"):
    if l.startswith("userID"):
    #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    y=0
    #if type(usercategorydic[u])!=int and type(categorydic[i])!=int:
    for cat in (categorydic[i]):
        for j in range(len(usercategorydic[u])):
            if cat in usercategorydic[u][j]:
                 y=1
    if y==1:
        predictions.write(u + '-' + i + ",1\n")
    else:
        predictions.write(u + '-' + i + ",0\n")
    c+=1

predictions.close()

My Kaggle name is Chao Yu

In [402]:
####### Q5
import pandas as pd
sumofrating=0
for l in trainset:
    rating=l['rating']
    sumofrating+=rating
alpha=sumofrating/len(trainset)
print "alpha is",alpha

alpha is 4.18703


In [403]:
import math
MSE=0
for l in validationset:
    rating=l['rating']
    MSE+=(alpha-rating)**2
MSE=MSE/len(validationset)
print "MSE on the validation set is",MSE

MSE on the validation set is 0.748343744499


In [404]:
businesslist = []
userlist=[]
for l in validationset:
    user,business = l['userID'],l['businessID']
    businesslist.append(business)
    userlist.append(user)
businesslist=list(set(businesslist))  
userlist=list(set(userlist))    

In [448]:
#######Q6
df=pd.DataFrame(validationset)
userRatings_train = defaultdict(lambda: defaultdict(int))
businessRatings_train = defaultdict(lambda: defaultdict(int))
beta_u = defaultdict(float)
beta_i = defaultdict(float)
gamma_u= defaultdict(float)
gamma_i= defaultdict(float)

for l in data:
    user,business,rating = l['userID'],l['businessID'],l['rating']
    userRatings_train[user][business] = rating
    businessRatings_train[business][user] = rating
    #random initialization
    beta_u[user] = random.randint(1,100)/1000.0  
    beta_i[business] = random.randint(1,100)/1000.0
    gamma_u[user] = random.randint(1,100)/1000.0  
    gamma_i[business] = random.randint(1,100)/1000.0  


  

In [450]:
lamda = 1
iteration = 0
while iteration <= 100:
    # update alpha
    
    alpha=sum(df["rating"])
    for user in userRatings_train:
        for business in userRatings_train[user]:
            alpha -= (beta_u[user] + beta_i[business])
    alpha /= len(trainset)
    # update beta_u
    for user in userRatings_train:
        beta_u[user] = 0
        for business in userRatings_train[user]:
            beta_u[user] += userRatings_train[user][business] \
                    - (alpha + beta_i[business])
        beta_u[user] /= (lamda + len(userRatings_train[user])) 
    # update beta_i
    for business in businessRatings_train:
        beta_i[business] = 0
        for user in businessRatings_train[business]:
            beta_i[business] += businessRatings_train[business][user] \
                    - (alpha + beta_u[user])
        beta_i[business] /= (lamda + len(businessRatings_train[business]))
    iteration += 1

In [409]:
### calculate MSE on validation set
userRatings_valid = defaultdict(lambda: defaultdict(int))
businessRatings_valid = defaultdict(lambda: defaultdict(int))

for l in validationset:
    user,business,rating = l['userID'],l['businessID'],l['rating']
    userRatings_valid[user][business] = rating
    businessRatings_valid[business][user] = rating
MSE = 0
for user in userRatings_valid:
    for business in userRatings_valid[user]:
        x=alpha
        if user in beta_u:
            x+=beta_u[user]
        if business in beta_i:
            x+=beta_i[business]
        MSE+=((x-userRatings_valid[user][business])**2)

MSE /= len(validationset)
print "MSE on validation set",MSE


MSE on validation set 2.00767269453e+55


In [36]:
####Q7
import operator
sorted_beta_u=sorted(beta_u.items(), key=operator.itemgetter(1))
sorted_beta_i=sorted(beta_i.items(), key=operator.itemgetter(1))
print "user id with maximum beta is",sorted_beta_u[-1][0]
print "maximum beta is",sorted_beta_u[-1][1]
print "user id with minimum beta is",sorted_beta_u[0][0]
print "minimum beta is",sorted_beta_u[0][1]
print "business id with maximum beta is",sorted_beta_i[-1][0]
print "maximum beta is",sorted_beta_i[-1][1]
print "business id with minimum beta is",sorted_beta_i[0][0]
print "minimum beta is",sorted_beta_i[0][1]

user id with maximum beta is U357799541
maximum beta is 1.1793783493
user id with minimum beta is U417838537
minimum beta is -2.80365895513
business id with maximum beta is B093985406
maximum beta is 1.18311575064
business id with minimum beta is B241777680
minimum beta is -2.21741811964


In [410]:
lamdaset=[0.01,0.1,1,10,100,1000,10000]
for lamda in lamdaset:
    iteration = 0
    while iteration <= 100:
    # update alpha
        alpha=sum(df["rating"])
        for user in userRatings_train:
            for business in userRatings_train[user]:
                alpha -= (beta_u[user] + beta_i[business])
        alpha /= len(trainset)
        # update beta_u
        for user in userRatings_train:
            beta_u[user] = 0
            for business in userRatings_train[user]:
                beta_u[user] += userRatings_train[user][business] \
                        - (alpha + beta_i[business])
            beta_u[user] /= (lamda + len(userRatings_train[user])) 
        # update beta_i
        for business in businessRatings_train:
            beta_i[business] = 0
            for user in businessRatings_train[business]:
                beta_i[business] += businessRatings_train[business][user] \
                        - (alpha + beta_u[user])
            beta_i[business] /= (lamda + len(businessRatings_train[business]))
        # MSE
        MSE = 0
        for user in userRatings_train:
            for business in userRatings_train[user]:
                MSE += (alpha + beta_u[user] + beta_i[business] 
                        - userRatings_train[user][business]) **2
        MSE /= len(trainset)
        if iteration==100:
            print "lamda is", lamda
            print "MSE for training set is", MSE
        iteration+=1
    ### calculate MSE on validation set
    userRatings_valid = defaultdict(lambda: defaultdict(int))
    businessRatings_valid = defaultdict(lambda: defaultdict(int))

    for l in validationset:
        user,business,rating = l['userID'],l['businessID'],l['rating']
        userRatings_valid[user][business] = rating
        businessRatings_valid[business][user] = rating
    MSE = 0
    for user in userRatings_valid:
        for business in userRatings_valid[user]:
            x=alpha
            if user in beta_u:
                x+=beta_u[user]
            if business in beta_i:
                x+=beta_i[business]
            MSE+=(x-userRatings_valid[user][business])**2

    MSE /= len(validationset)
    print "MSE for validation set is",MSE
  

lamda is 8
MSE for training set is 2.16645929841e+100
MSE for validation set is 1.08365624909e+100
lamda is 9
MSE for training set is 3.52313312698e+141
MSE for validation set is 1.76206437062e+141


KeyboardInterrupt: 

We can see when lamda is 10, the MSE for validation set is lowest, which is 0.621537959215.

In [38]:
lamda = 10
iteration = 0
while iteration <= 100:
    # update alpha
    
    alpha=sum(df["rating"])
    for user in userRatings_train:
        for business in userRatings_train[user]:
            alpha -= (beta_u[user] + beta_i[business])
    alpha /= len(trainset)
    # update beta_u
    for user in userRatings_train:
        beta_u[user] = 0
        for business in userRatings_train[user]:
            beta_u[user] += userRatings_train[user][business] \
                    - (alpha + beta_i[business])
        beta_u[user] /= (lamda + len(userRatings_train[user])) 
    # update beta_i
    for business in businessRatings_train:
        beta_i[business] = 0
        for user in businessRatings_train[business]:
            beta_i[business] += businessRatings_train[business][user] \
                    - (alpha + beta_u[user])
        beta_i[business] /= (lamda + len(businessRatings_train[business]))
    # MSE
    MSE = 0
    for user in userRatings_train:
        for business in userRatings_train[user]:
            MSE += (alpha + beta_u[user] + beta_i[business] 
                    - userRatings_train[user][business]) **2
    MSE /= len(trainset)
    iteration += 1


In [39]:
predictions = open("predictions_Rating.txt", 'w')
for l in open("pairs_Rating.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    x=alpha
    if u in beta_u:
        x += beta_u[u]
    if i in beta_i:
        x += beta_i[i] 
        
    predictions.write(u + '-' + i + ',' + str(x) + '\n')
    
predictions.close()

In [40]:
userRatings = defaultdict(lambda: defaultdict(int))
businessRatings = defaultdict(lambda: defaultdict(int))
beta_u = defaultdict(float)
beta_i = defaultdict(float)


for l in data:
    user,business,rating = l['userID'],l['businessID'],l['rating']
    userRatings[user][business] = rating
    businessRatings[business][user] = rating
    #random initialization
    beta_u[user] = random.randint(1,100)/1000.0  
    beta_i[business] = random.randint(1,100)/1000.0

In [41]:
###### Use all data to make prediction
df=pd.DataFrame(data)
lamda = 10
iteration = 0
while iteration <= 100:
    # update alpha
    
    alpha=sum(df["rating"])
    for user in userRatings:
        for business in userRatings[user]:
            alpha -= (beta_u[user] + beta_i[business])
    alpha /= len(data)
    # update beta_u
    for user in userRatings:
        beta_u[user] = 0
        for business in userRatings[user]:
            beta_u[user] += userRatings[user][business] \
                    - (alpha + beta_i[business])
        beta_u[user] /= (lamda + len(userRatings[user])) 
    # update beta_i
    for business in businessRatings:
        beta_i[business] = 0
        for user in businessRatings[business]:
            beta_i[business] += businessRatings[business][user] \
                    - (alpha + beta_u[user])
        beta_i[business] /= (lamda + len(businessRatings[business]))
    iteration += 1

In [42]:
predictions = open("predictions_Rating.txt", 'w')
for l in open("pairs_Rating.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    x=alpha
    if u in beta_u:
        x += beta_u[u]
    if i in beta_i:
        x += beta_i[i] 
        
    predictions.write(u + '-' + i + ',' + str(x) + '\n')
    
predictions.close()